# DBSCAN

In this notebook you will use GPU-accelerated DBSCAN to identify clusters of infected people.

## Objectives

By the time you complete this notebook you will be able to:

- Use GPU-accelerated DBSCAN
- Use cuXfilter to visualize DBSCAN clusters

## Imports

In [1]:
import cudf
import cuml

import cuxfilter as cxf

## Load Data

For this notebook, we again load a subset of our population data with only the columns we need. An `infected` column has been added to the data to indicate whether or not a person is known to be infected with our simulated virus.

In [2]:
gdf = cudf.read_csv('./data/pop_2-04.csv', dtype=['float32', 'float32', 'float32'])
print(gdf.dtypes)
gdf.shape

northing    float32
easting     float32
infected    float32
dtype: object


(1000000, 3)

In [3]:
gdf.head()

,northing,easting,infected
0,178547.296875,368012.1250,0.0
1,174068.281250,543802.1250,0.0
2,358293.687500,435639.8750,0.0
3,87240.304688,389607.3750,0.0
4,158261.015625,340764.9375,0.0


In [4]:
gdf['infected'].value_counts()

0.0    984331
1.0     15669
Name: infected, dtype: int32

## DBSCAN Clustering

DBSCAN is another unsupervised clustering algorithm that is particularly effective when the number of clusters is not known up front and the clusters may have concave or other unusual shapes--a situation that often applies in geospatial analytics.

In this series of exercises you will use DBSCAN to identify clusters of infected people by location, which may help us identify groups becoming infected from common patient zeroes and assist in response planning.

## Exercise: Make a DBSCAN Instance

Create a DBSCAN instance by using `cuml.DBSCAN`. Pass in the named argument `eps` (the maximum distance a point can be from the nearest point in a cluster to be considered possibly in that cluster) to be `5000`. Since the `northing` and `easting` values we created are measured in meters, this will allow us to identify clusters of infected people where individuals may be separated from the rest of the cluster by up to 5 kilometers.

In [6]:
dbscan = cuml.DBSCAN(eps=13000, min_samples = 80)

#### Solution

In [5]:
# %load solutions/dbscan_instance
dbscan = cuml.DBSCAN(eps=5000)


### Exercise: Identify Infected Clusters

Create a new dataframe from rows of the original dataframe where `infected` is `1` (true), and call it `infected_df`--be sure to reset the dataframe's index afterward. Use `dbscan.fit_predict` to perform clustering on the `northing` and `easting` columns of `infected_df`, and turn the resulting series into a new column in `infected_gdf` called "cluster". Finally, compute the number of clusters identified by DBSCAN.

In [8]:
infected_df = gdf[gdf['infected'] == 1].reset_index()
infected_df['cluster'] = dbscan.fit_predict(infected_df[['northing', 'easting']])
infected_df['cluster'].nunique()

12

#### Solution

In [7]:
# %load solutions/identify_infected
infected_df = gdf[gdf['infected'] == 1].reset_index()
infected_df['cluster'] = dbscan.fit_predict(infected_df[['northing', 'easting']])
infected_df['cluster'].nunique()


In [9]:
infected_df.groupby('cluster')['index'].count()

cluster
 7      165
 8       96
 0     4355
 1     3776
 6      260
 2      356
 5      341
-1     4774
 4      907
 9      115
 10     128
 3      396
Name: index, dtype: int32

## Visualize the Clusters

Because we have the same column names as in the K-means example--`easting`, `northing`, and `cluster`--we can use the same code to visualize the clusters.

### Associate a Data Source with cuXfilter

In [ ]:
cxf_data = cxf.DataFrame.from_dataframe(infected_df)

### Define Charts and Widgets

As in the K-means notebook, we have an existing integer column to use with multi-select: `cluster`.

In [12]:
from pyproj import Proj, Transformer

transform_27700_to_3857 = Transformer.from_crs('epsg:27700', 'epsg:3857')
infected_df['dropoff_y'], infected_df['dropoff_x'] = transform_27700_to_3857.transform(infected_df['northing'].values_host, infected_df['easting'].values_host)
infected_df['dropoff_x'] = infected_df['dropoff_x'] - 7300000
infected_df['dropoff_y'] = infected_df['dropoff_y'] + 7300000
infected_df

,index,northing,easting,infected,cluster,dropoff_y,dropoff_x
0,41,595509.250000,426545.59375,1.0,-1,7.406878e+06,-186691.754862
1,182,185086.609375,547718.00000,1.0,0,6.705186e+06,18944.344419
2,190,183406.546875,528708.06250,1.0,0,6.703855e+06,-14041.976466
3,216,373846.562500,485603.84375,1.0,-1,7.032495e+06,-79413.103937
4,266,194947.406250,535378.37500,1.0,0,6.723192e+06,-1602.500512
...,...,...,...,...,...,...,...
15664,999847,321325.937500,401268.15625,1.0,1,6.945161e+06,-223560.124847
15665,999947,181178.015625,526044.25000,1.0,0,6.700242e+06,-18818.198797
15666,999963,236423.734375,602746.68750,1.0,-1,6.790423e+06,118664.723782
15667,999979,186677.390625,517625.53125,1.0,0,6.710377e+06,-32862.500640


In [13]:
import cuxfilter
from bokeh.tile_providers import get_provider
from matplotlib import pyplot as plt

palette = ["red", "grey", "black", "purple", "pink", "yellow", "brown", "green", "orange", "blue"]
chart1 = cuxfilter.charts.scatter(x='dropoff_x', y='dropoff_y', aggregate_col='cluster', aggregate_fn='mean',
                                                    tile_provider="CartoLight", color_palette=palette, pixel_shade_type='linear')
chart2 = cuxfilter.charts.scatter(
        x='dropoff_x', y='dropoff_y',
        point_size=4,
        pixel_shade_type='linear',
        pixel_spread='spread',
        title='Scatter Layout'
      )

cluster_widget = cuxfilter.charts.panel_widgets.multi_select('cluster')

In [19]:
cxf_data = cxf.DataFrame.from_dataframe(infected_df)

### Create and Show the Dashboard

In [20]:
from cuxfilter.layouts import feature_and_double_base

dash = cxf_data.dashboard([chart1, chart2], sidebar=[cluster_widget], theme=cuxfilter.themes.rapids, title= 'infected DATASET')
dash.app()

Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=280)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=15669)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='cluster_multi_select', width=400)
            [0] MultiSelect(height=200, options=[-1, 0, 1, 2, ...], sizing_mode='stretch_both', value=[''], width=400)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=11, nrows=6, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='Scatter plot f..., width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')

In [21]:

chart1.view()

Card(sizing_mode='scale_width', title='Scatter plot f..., width=800)
    [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')

In [22]:
%%js
var host = window.location.host;
element.innerText = "'http://"+host+"'";

<IPython.core.display.Javascript object>

Set `my_url` in the next cell to the value just printed, making sure to include the quotes:

In [23]:
my_url = 'http://ec2-34-207-143-122.compute-1.amazonaws.com'# TODO: Set this value to the print out of the cell above, including the quotes.
dash.show(my_url, port=8789)

Dashboard running at port 8789


Row(sizing_mode='stretch_both')
    [0] Button(button_type='success', name='open cuxfilter d..., sizing_mode='stretch_both')

... and you can run the next cell to generate a link to the dashboard:

In [24]:
%%js
var host = window.location.host;
var url = 'http://'+host+'/lab/proxy/8789/';
element.innerHTML = '<a style="color:blue;" target="_blank" href='+url+'>Open Dashboard</a>';

<IPython.core.display.Javascript object>

In [ ]:
dash.stop()

<br>
<div align="center"><h2>Please Restart the Kernel</h2></div>

In [25]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Next

In the next notebook, you will use GPU-accelerated logistic regression to estimate infection risk based on features of our population members.